## 用户关注分组数据预处理脚本
* 用户分组名比较自由、口语化，包含许多emoji、特殊符号等，因此先做预处理，方便词向量文件中筛选出


### 背景及整体流程
1. 首先获取用户分组与频次数据(101机器：/data0/ad_dm/online/weibo_crowds_mining/data/group_name_list.dat)
2. 同步数据到70上，取top1w做调研
3. 使用本脚本做预处理:去除emoji/数字/特殊符号等
4. 使用简单、暴力的方案筛选向量 grep -w -F -f group_chn_name_1w.dat w2v_skip_gram.txt  > group_chn_name_1w.vector
5. 文件中仍然有类似“马克．吐温”的词，使用awk过滤：awk -F ' ' '{if(!match($1,"·")){print $0}}'
6. 之后交由script/w2c_tsne_visualize.ipynb 脚本做可视化
7. 最终分析关注动机，能为“关系类型”确定带来启发以及之后如何利用分组名称

In [90]:
import re

#####定义正则匹配相关词的格式
digit_pattern = re.compile(ur"[0-9]+")
chn_pattern = re.compile(ur"[\u4e00-\u9fff]")
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)
special_pattern=re.compile("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+".decode("utf8"))
regexp = re.compile(r'[^\x00-\x7f]')  ###非英文ASCII

In [91]:
###匹配汉字且不带表情、数字等
def get_chn_line(input_file,output_file):
    f_output=open(output_file,'w+')
    count=0
    for line in open(input_file):
        line  = line.strip().decode("utf8")
        if chn_pattern.search(line) and not emoji_pattern.search(line) and not digit_pattern.search(line) and not special_pattern.search(line):
           ## print(line)
            print >> f_output,"%s" % line.encode("utf8")
            count=count+1
    print(count)

###含有表情的
def get_emoji_line(file):
    count=0
    for line in open(file):
        line  = line.strip().decode("utf8")
        if emoji_pattern.search(line):
            print(line)
            count=count+1
    print(count)
###含有数字的
def get_digit_line(file):
    count=0
    for line in open(file):
        line  = line.strip().decode("utf8")
        if digit_pattern.search(line):
            print(line)
            count=count+1
    print(count)
###含有特殊符号的
def get_special_line(file):
    count=0
    for line in open(file):
        line  = line.strip().decode("utf8")
        if special_pattern.search(line):
            print(line)
            count=count+1
    print(count)

In [92]:
####测试各正则匹配的效果
#get_emoji_line("/data1/ad_dm/rsync_data/group_pure_name_5w.dat")
#get_digit_line("/data1/ad_dm/rsync_data/group_pure_name_5w.dat")
#get_special_line("/data1/ad_dm/rsync_data/group_pure_name_5w.dat")

In [93]:
get_chn_line("../data/group_chn_name_1w.dat","../data/group_chn_name.data")

10000
